## Value-at-Risk: Forecast Evaluation

**Functions**

`sm.OLS`, `stats.bernoulli`

### Exercise 61
Evaluate the FHS and HS VaR forecasts constructed in the previous exercises using:

* HIT tests
* The Bernoulli test for unconditionally correct VaR
* Christoffersen’s test for conditionally correct VaR


In [ ]:
import pandas as pd

# Load Data
sp500 = pd.read_hdf("./data/arch-data.h5", "sp500")        
eurusd = pd.read_hdf("./data/arch-data.h5", "eurusd")

sp500_returns = 100 * sp500.SP500.pct_change().dropna()
eurusd_returns = 100 * eurusd.DEXUSEU.pct_change().dropna()

with pd.HDFStore("./data/hs-var.h5", mode="r") as hdf:
    sp500_hs = hdf.get("sp500_var")
    eurusd_hs = hdf.get("eurusd_var")
with pd.HDFStore("./data/fhs-var.h5", mode="r") as hdf:
    sp500_fhs = hdf.get("sp500_var")
    eurusd_fhs = hdf.get("eurusd_var")
# Construct HITS

In [ ]:
start = int(0.25 * sp500_returns.shape[0])
sp500_fhs.head()
multi_day = {}
for h in (1, 5, 10):
    multi_day[f"horizon_{h}"] = sp500_returns.rolling(h).sum()
multi_day = pd.DataFrame(multi_day)

In [ ]:
sp500_hs.columns=["hs_1","hs_5","hs_10"]
sp500_fhs.columns=["fhs_1","fhs_5","fhs_10"]
combined = pd.concat([multi_day, sp500_hs, sp500_fhs], axis=1)
combined = combined.dropna(how="any")

In [ ]:
hits = {"hit_hs_1": combined.horizon_1 < -combined.hs_1,
        "hit_hs_5": combined.horizon_5 < -combined.hs_5,
        "hit_hs_10": combined.horizon_10 < -combined.hs_10,
        "hit_fhs_1": combined.horizon_1 < -combined.fhs_1,
        "hit_fhs_5": combined.horizon_5 < -combined.fhs_5,
        "hit_fhs_10": combined.horizon_10 < -combined.fhs_10}
hits = pd.DataFrame(hits).dropna(how="all")
hits.head(10)

In [ ]:
import numpy as np
import statsmodels.api as sm
# TODO: Be careful about lags in multistep and what is known
for col in hits:
    data = hits[col] - 0.05
    key = col.replace("hit_","")
    data_with_fcast = pd.concat([data, combined[key]],axis=1).astype(np.double)

    x, y = sm.tsa.lagmat(data_with_fcast[col], 10, trim="both", original="sep", use_pandas=True)
    x = pd.concat([combined[key], x],axis=1)
    x = sm.add_constant(x)
    x = x.dropna()
    res = sm.OLS(y, x).fit(cov_type="HAC",cov_kwds={"maxlags":int(1.2*x.shape[0]**(2/5))})
    test_statistic = res.wald_test(np.eye(12), use_f=False)
    print(test_statistic)
    print(res.summary())